In [24]:
import pandas as pd
# for loading resource from local machine (all .csv files should already be placed in current folder)
local_tables = ["ca_ev_registrations_public.csv",
                "co_ev_registrations_public.csv",
                "ct_ev_registrations_public.csv",
                "fl_ev_registrations_public.csv",
                "mt_ev_registrations_public.csv",
                "mi_ev_registrations_public.csv",
                "mn_ev_registrations_public.csv",
                "nj_ev_registrations_public.csv",
                "ny_ev_registrations_public.csv",
                "or_ev_registrations_public.csv",
                "tn_ev_registrations_public.csv",
                "tx_ev_registrations_public.csv",
                "vt_ev_registrations_public.csv",
                "va_ev_registrations_public.csv",
                "wa_ev_registrations_public.csv",
                "wi_ev_registrations_public.csv",]
# for loading resource from Web (some .csv files are too big (> 200 M) to be stored Github repo)
tables = ["https://www.atlasevhub.com/public/dmv/ca_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/co_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/ct_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/fl_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/mt_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/mi_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/mn_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/nj_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/ny_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/or_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/tn_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/tx_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/vt_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/va_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/wa_ev_registrations_public.csv",
          "https://www.atlasevhub.com/public/dmv/wi_ev_registrations_public.csv",]
states = ["California", "Colorado", "Connecticut", "Florida", "Montana", "Michigan", "Minnesota", "New Jersey", 
          "New York", "Oregon", "Tennessee", "Texas", "Vermont", "Virginia", "Washington", "Wisconsin"]
EV_registration_columns = ["State", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]
EV_registration = pd.DataFrame(columns = EV_registration_columns)
EV_model_cloumns = ["State", "Year", "Model", "Amount"]
EV_model = pd.DataFrame(columns = EV_model_cloumns)
# process data from each state (16 in total)
for i in range(16):
    curr_state = states[i]
    curr_table = tables[i]
    table = pd.read_csv(curr_table, low_memory=False)
    table["Registration Valid Date"] = pd.to_datetime(table["Registration Valid Date"])
    table["Registration Valid Date"] = table["Registration Valid Date"].dt.year
    table = table[table['Registration Valid Date'].notna()]
    table["Registration Valid Date"].astype(int).dtypes
    table = table[(table["Registration Valid Date"] >= 2011) & (table["Registration Valid Date"] <= 2018)]
    table = table.sort_values(by='Registration Valid Date')
    annual_registration_amount = table["Registration Valid Date"].value_counts(sort = False).tolist()
    # ONLY process states having data for all 8 years (2011-2018) 
    if len(annual_registration_amount) == 8:
        # add new row (representing the annual number of EV registered in current state) to table "EV_registration" 
        newrow_data = [curr_state] + annual_registration_amount
        newrow_df = pd.DataFrame([newrow_data],columns = EV_registration_columns) 
        EV_registration = pd.concat([EV_registration, newrow_df], axis='rows', ignore_index=True)
        for j in range(8):
            # add new row (representing the annual amount of EV (by model) registered in current state) to table "EV_model" 
            curr_year = 2011 + j
            curr_year_table = table[table['Registration Valid Date'] == curr_year]
            if 'Vehicle Name' not in table.columns:
                curr_year_table = curr_year_table.assign(Vehicle_Name = lambda x: x.Make + x.Model)
                curr_year_table = curr_year_table.rename(columns={"Vehicle_Name": "Vehicle Name"})
            curr_year_model = curr_year_table["Vehicle Name"].value_counts(sort = False).keys().tolist()
            curr_year_model_amount = curr_year_table["Vehicle Name"].value_counts(sort = False).tolist()
            model_amount = len(curr_year_table["Vehicle Name"].unique())
            for k in range(model_amount):
                curr_model = curr_year_model[k]
                curr_model_amount = curr_year_model_amount[k]
                newrow_data = [curr_state,curr_year,curr_model,curr_model_amount]
                newrow_df = pd.DataFrame([newrow_data],columns = EV_model_cloumns) 
                EV_model = pd.concat([EV_model, newrow_df], axis='rows', ignore_index=True)
    else:
        print(curr_state + " only has data in " + str(table["Registration Valid Date"].unique()) + " among the period 2011-2018")
print("------------------------------------------------------------------------------------------------------------------------")
print(EV_registration)
print("------------------------------------------------------------------------------------------------------------------------")
print(EV_model)

Connecticut only has data in [2011 2012 2013 2014 2015 2016 2018] among the period 2011-2018
Florida only has data in [2018] among the period 2011-2018
Michigan only has data in [2013 2014 2015 2016 2017 2018] among the period 2011-2018
New Jersey only has data in [2017 2018] among the period 2011-2018
Oregon only has data in [] among the period 2011-2018
Tennessee only has data in [] among the period 2011-2018
Texas only has data in [] among the period 2011-2018
Wisconsin only has data in [2018] among the period 2011-2018
------------------------------------------------------------------------------------------------------------------------
        State  2011   2012   2013    2014    2015    2016    2017    2018
0  California  5857  18356  52427  108939  166168  230171  322762  448567
1    Colorado   308   1046   2529    4129    6062    8958   13180   18020
2     Montana    10     18     28      43      52      59     132     866
3   Minnesota    12     28     67      77     148    1